In [1]:
#====================================
# Author: gmd8si
# Date: 16, November, 2016
#====================================
%matplotlib inline
import tensorflow as tf
import numpy as np
import pandas as pd
from detector_dr import Detector
from util import load_image, crop_image, augment           # Questionalable since we can use our own loading pipeline
import os
import matplotlib.pyplot as plt
import csv
import time
#================================================
# Setting Hyper-Parameters 
#================================================

#weight_path = '../data/caffe_layers_value.pickle'
#model_path = '../models/caltech256/'
PRETRAINED_MODEL_PATH='/home/gmd8si/Weakly_detector/src/ckpt_dir/model-customVGGCAM128oldGRAD.ckpt-20' #None 
#'/home/gmd8si/Weakly_detector/src/ckpt_dir/model.ckpt-0'
N_EPOCHS = 10000
INIT_LEARNING_RATE = 0.01
WEIGHT_DECAY_RATE = 0.0005
MOMENTUM = 0.9
IMAGE_HEIGHT  = 128    #960
IMAGE_WIDTH   = 128    #720
NUM_CHANNELS  = 3
BATCH_SIZE = 100
N_LABELS = 5
DROPOUT = 0.50  
LOGS_PATH = './tensorflow_logs/custom256'
WEIGHT_PATH = None
TRAINSET_PATH = '/home/gmd8si/Weakly_detector/data/customKaggle128_train.csv'
VALSET_PATH ='/home/gmd8si/Weakly_detector/data/customKaggle128_val.csv'

# Reading Data

In [2]:
#=======================================================================================================
# Reading Training data from CSV FILE
#=======================================================================================================

csv_path = tf.train.string_input_producer([TRAINSET_PATH], shuffle=True)
textReader = tf.TextLineReader()
_, csv_content = textReader.read(csv_path)
im_name, im_label = tf.decode_csv(csv_content, record_defaults=[[""], [1]])

im_content = tf.read_file(im_name)
train_image = tf.image.decode_jpeg(im_content, channels=3)
#train_image = crop_image(train_image)   # Will do it again
train_image = tf.cast(train_image, tf.float32) / 255. # could be unnecessary
train_image = augment(train_image)
size = tf.cast([IMAGE_HEIGHT, IMAGE_WIDTH], tf.int32)
train_image = tf.image.resize_images(train_image, size)
#label = tf.one_hot(label, 5, 1, 0 )
train_label = tf.cast(im_label, tf.int64) # unnecessary
train_image_batch, train_label_batch = tf.train.shuffle_batch([train_image, train_label], batch_size=BATCH_SIZE,
                                                             capacity = 4000 + 3*BATCH_SIZE, min_after_dequeue = 4000)

In [3]:
#=======================================================================================================
# Reading Validation data from CSV FILE
#=======================================================================================================

val_csv_path = tf.train.string_input_producer([VALSET_PATH], shuffle=True) # No need to set no. epochs here
val_textReader = tf.TextLineReader()
_, val_content = val_textReader.read(val_csv_path)
val_image, val_label = tf.decode_csv(val_content, record_defaults=[[""], [1]])

val_image_content = tf.read_file(val_image)
val_image = tf.image.decode_jpeg(val_image_content, channels=3)
val_image = tf.cast(val_image, tf.float32) / 255. # could be unnecessary
size = tf.cast([IMAGE_HEIGHT, IMAGE_WIDTH], tf.int32)
val_image = tf.image.resize_images(val_image, size)
val_label = tf.cast(val_label, tf.int64) # unnecessary
val_image_batch, val_label_batch = tf.train.batch([val_image, val_label], batch_size=BATCH_SIZE)

# Determining datasets length

In [4]:
trainfile = open(TRAINSET_PATH, "rb")
reader = csv.reader(trainfile)
trainset = sum(1 for row in reader)
print trainset
trainfile.close()

valfile = open(VALSET_PATH, "rb")
reader = csv.reader(valfile)
valset = sum(1 for row in reader)
print valset
valfile.close()

4584
830


# Placeholder

In [5]:
learning_rate = tf.placeholder( tf.float32, [])
images_tf = tf.placeholder( tf.float32, [None, IMAGE_HEIGHT, IMAGE_WIDTH, 3], name="images")
labels_tf = tf.placeholder( tf.int64, [None], name='labels')  # the dimensions could be [None,N_CLASSES]

# Optimizer Properties

In [6]:
detector = Detector(N_LABELS)

In [7]:
p1,p2,p3,p4,conv5, conv6, gap, output = detector.inference(images_tf, DROPOUT, train=True)
with tf.name_scope('Loss'):
    loss_tf = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits( output, labels_tf ), name='loss_tf')
    loss_summary = tf.scalar_summary("loss", loss_tf)


In [8]:
#weights_only = filter( lambda x: x.name.endswith('W:0'), tf.trainable_variables() )
#weight_decay = tf.reduce_sum(tf.pack([tf.nn.l2_loss(x) for x in weights_only])) * WEIGHT_DECAY_RATE
#loss_tf += weight_decay

In [9]:
#train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss_tf)
#train_op = tf.train.MomentumOptimizer(learning_rate, MOMENTUM).minimize(loss_tf)
train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss_tf)
'''
optimizer = tf.train.MomentumOptimizer( learning_rate, MOMENTUM)
grads_and_vars = optimizer.compute_gradients( loss_tf )
grads_and_vars = map(lambda gv: (gv[0], gv[1]) if ('conv6' in gv[1].name or 'GAP' in gv[1].name) else (gv[0]*0.1, gv[1]), 
                     grads_and_vars)
grads_and_vars = [(tf.clip_by_value(gv[0], -5., 5.), gv[1]) for gv in grads_and_vars]
train_op = optimizer.apply_gradients( grads_and_vars )
'''

"\noptimizer = tf.train.MomentumOptimizer( learning_rate, MOMENTUM)\ngrads_and_vars = optimizer.compute_gradients( loss_tf )\ngrads_and_vars = map(lambda gv: (gv[0], gv[1]) if ('conv6' in gv[1].name or 'GAP' in gv[1].name) else (gv[0]*0.1, gv[1]), \n                     grads_and_vars)\ngrads_and_vars = [(tf.clip_by_value(gv[0], -5., 5.), gv[1]) for gv in grads_and_vars]\ntrain_op = optimizer.apply_gradients( grads_and_vars )\n"

In [10]:
# Histogram Summaries
# Add histograms for trainable variables.
for var in tf.trainable_variables():
    tf.histogram_summary(var.op.name, var)

# Add histograms for gradients.
#for grad, var in grads_and_vars:
#    if grad is not None:
#        tf.histogram_summary(var.op.name + '/gradients', grad)

In [11]:
summary_op = tf.merge_all_summaries()

# Accuracy

In [12]:
#label_predictions = output.argmax(axis=1)
#accuracy = (label_predictions == labels_tf).sum()
# check if we have access to labels_tf in session, like the train.caltech

correct_pred = tf.equal(tf.argmax(output, 1), labels_tf)
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


# Tensorflow checkpoint saver

In [13]:
# Saver Operation to save and restore all variables, first create directory
ckpt_dir = "./ckpt_dir"
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)
saver = tf.train.Saver(max_to_keep=50)   #Indicates max number of recent checkpoint files to keep

In [14]:
if not os.path.exists('./results'):
    os.makedirs('./results')
f_log = open('./results/customVGGCAM128n.txt', 'w')


# Training and Validation Sessions

In [ ]:
with tf.Session() as sess:
    
    if PRETRAINED_MODEL_PATH:
        print "Using Pretrained model"
        saver.restore(sess, PRETRAINED_MODEL_PATH)
    else:    
        sess.run(tf.initialize_all_variables())    
    #sess.run(tf.initialize_local_variables())
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    loss_list = []
    process_start_time= time.time()
    summary_writer = tf.train.SummaryWriter(LOGS_PATH, graph=tf.get_default_graph())
    steps = 1
    count = 1
    
    for epoch in range(300):    
        
        train_correct = 0
        train_data = 0    
        epoch_start_time = time.time()
        
        for i in range(trainset/BATCH_SIZE +1):
            train_imbatch, train_labatch = sess.run([train_image_batch, train_label_batch])
            _, loss_val, output_val, off_acc, summary = sess.run([train_op, loss_tf, output, accuracy, summary_op], feed_dict={
                    learning_rate: INIT_LEARNING_RATE, images_tf: train_imbatch, labels_tf: train_labatch})
                
            loss_list.append(loss_val)                              # This must be for visualizing loss curve 
            
            label_predictions = output_val.argmax(axis=1)
            acc = (label_predictions == train_labatch).sum()
            
            train_correct +=acc               #No of correctly classified examples, accumulated per batch
            train_data += len(output_val)
            
            if (steps) % 5 == 0:   # after 5 batches
                print "======================================"
                print "Epoch", epoch+1, "Iteration", steps
                print "Processed", train_data, '/', trainset               # (count*BATCH_SIZE)
                batch_predictions = output_val.argmax(axis=1)
                batch_acc = (batch_predictions == train_labatch).sum()
                print 'official accuracy: ', off_acc
                print 'Training accuracy (correctly classified/num of examples) = ', (train_correct/float(train_data))
                print "Accuracy per batch:", batch_acc
                #print "Accuracy Percentage:", (acc/len(current_labels))*100
                print "Training Loss:", np.mean(loss_list)
                loss_list = []
                summary_writer.add_summary(summary, steps)
            steps += 1
            count += 1
        count = 1
        n_correct = 0
        n_data = 0
        
        
        for i in range (valset/BATCH_SIZE +1):
            
            val_imbatch, val_labatch = sess.run([val_image_batch, val_label_batch])
            output_vals = sess.run(output, feed_dict={images_tf:val_imbatch, labels_tf: val_labatch})
            
            # check the length of output_vals, and no. of values in n_data, correct the processed value as well
            label_predictions = output_vals.argmax(axis=1)
            acc = (label_predictions == val_labatch).sum()
        
            n_correct += acc                  # No of correctly idenified examples in whole val set
            n_data += BATCH_SIZE
            

        acc_all = n_correct / float(n_data)
        f_log.write('epoch:'+str(epoch+1)+'\tacc:'+str(acc_all) + '\n')
        print "===========**VALIDATION ACCURACY**================"
        print 'epoch:'+str(epoch+1)+'\tacc:'+str(acc_all) + '\n'
        print 'Time Elapsed for Epoch:'+str(epoch+1)+' is '+str ((time.time() - epoch_start_time)/60.)+' minutes'
        INIT_LEARNING_RATE *= 0.99
        
        if (epoch % 10 == 0):
            saver.save(sess, ckpt_dir + "/model-customVGGCAM128n.ckpt", global_step=epoch)
        

Using Pretrained model
Epoch 1 Iteration 5
Processed 500 / 4584
official accuracy:  0.24
Training accuracy (correctly classified/num of examples) =  0.22
Accuracy per batch: 24
Training Loss: 1.59772
Epoch 1 Iteration 10
Processed 1000 / 4584
official accuracy:  0.22
Training accuracy (correctly classified/num of examples) =  0.204
Accuracy per batch: 22
Training Loss: 1.60479
Epoch 1 Iteration 15
Processed 1500 / 4584
official accuracy:  0.34
Training accuracy (correctly classified/num of examples) =  0.230666666667
Accuracy per batch: 34
Training Loss: 1.58865
Epoch 1 Iteration 20
Processed 2000 / 4584
official accuracy:  0.37
Training accuracy (correctly classified/num of examples) =  0.252
Accuracy per batch: 37
Training Loss: 1.57723
Epoch 1 Iteration 25
Processed 2500 / 4584
official accuracy:  0.2
Training accuracy (correctly classified/num of examples) =  0.2624
Accuracy per batch: 20
Training Loss: 1.57737
